# Tarea Árboles de desición

#### Equipo:

**Aide Jazmín González Cruz**

**Yusuri Arciga Reyes**

**Elena Villalobos Nolasco**

El objetivo de la última tarea consistirá en modificar el Jupyter notebook que se proporcionó con el código CaRT para que además de resolver problemas de clasificación sirva para resolver problemas de regresión. Probarlo con el set de datos de bateadores que se utilizó en clase para que puedan validar que está funcionando bien. Compararlo contra el resultado en R. Para ello habrá que importar el data frame de R en python y convertirlo en una lista de listas.

In [60]:
# Importando librerias necesarias
import pandas as pd
import numpy as np
import statistics

Cargando datos

In [61]:
hitters = pd.read_csv('Hitters.csv')
hitters.shape

(263, 21)

In [62]:
# Delete first column (names)
hitters.drop(hitters.columns[[0]], axis=1, inplace=True)
hitters.head(5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,6.163315,N
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,6.173786,A
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,6.214608,N
3,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,4.516339,N
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,6.620073,A


Si se usan todas las variables incluyendo las categóricas, estás serían algunos inputs en las funciones.

In [63]:
num_col_salary = 18
# Column labels.
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks","League" ,  
          "Division","PutOuts","Assists","Errors","Salary","NewLeague",
          "salary_c"]

In [64]:
# DEjando sólo las variable númericas
hitters.drop(['League'], axis=1, inplace=True)
hitters.drop(['Division'], axis=1, inplace=True)
hitters.drop(['NewLeague'], axis=1, inplace=True)
hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,6.163315
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,6.173786
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,6.214608
3,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,4.516339
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,6.620073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3,6.551080
259,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20,6.774224
260,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7,5.953243
261,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12,6.866933


In [65]:
quitar = ["AtBat","HmRun","Runs","RBI","Walks",  
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors"]
hits_years = hitters.copy()
hits_years = pd.DataFrame(hits_years.drop(quitar, axis=1))
hits_years.head(5)

,Hits,Years,Salary
0,81,14,6.163315
1,130,3,6.173786
2,141,11,6.214608
3,87,2,4.516339
4,169,11,6.620073


Pasando a Lista de Listas

- Para todas las variable númericas

In [66]:
hitters_lol = hitters.values.tolist()

- Sólo usando `hits` y `years`

In [67]:
hits_years_lol = hits_years.values.tolist()

## Funciones realizadas para variables continúas

In [68]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])
    #set regresa un conjunto({}) y regresa valores únicos

In [69]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [70]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [71]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [72]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [73]:
# Para clasificación
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

#RSS

In [74]:
# Caída del Gini
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [75]:
# Función para obtener promedios entre valores
def promedio(lista):
    #print(lista)
    prom = []
    for i in range(len(lista)-1):
        prom.append((lista[i]+lista[i+1])/2)
    return prom

In [76]:
# Extrae una columna de una lista de listas dada
def Extract(lst, col_num): 
    return [item[col_num] for item in lst] 

In [77]:
# Para regresión (Error error cuadrado medio para cortes en tarea)
def rss(prom, col, salary):
    """
    Calcula la suma de residuos al cuadrado (RSS)
    """
    #Saca medias
    mean_ant_div = []
    mean_post_div = []
    
    for div in range(len(prom)):
        ant_div = []
        post_div = []
        for i in range(len(col)):
            if (col[i] < prom[div]):#izq
                ant_div.append(salary[i])
    
            else:#der
                post_div.append(salary[i])
    
        mean_ant_div.append(statistics.mean(ant_div))
        mean_post_div.append(statistics.mean(post_div))
            
    # RSS
    rss_arr = []
    
    #RSS = suma (y-yhat)²    
    for div in range(len(prom)):
        _rss = 0.0
        
        for i in range(len(col)):
            if (col[i] < prom[div]):
                _rss = _rss + (salary[i] - mean_ant_div[div])**2                
            else:
                _rss = _rss + (salary[i] - mean_post_div[div])**2

        rss_arr.append(_rss)
        
    error_min = min(rss_arr)
    error_max = max(rss_arr)
    valor_corte = prom[rss_arr.index(min(rss_arr))]

    return error_min, error_max, valor_corte

In [78]:
#Aquí se juntan las funciones anteriores para correr cada columna
def errores(lol, num_col_salary):
    errores_min = []
    errores_max = []
    values_c = []
    error_min = 0
    error_max = 0
    valor = 0
    #print("lol: "+str(len(lol)))
    for i in range(len(lol[0])-1):
        #print("lol" + str(lol[i]))
        col = Extract(lol, i)
        #print("col" + str(col))
        if is_numeric(col[0]):
            #print("col" + str(col))
            Split_col = unique_vals(lol,i)
            Split_col = list(Split_col)
            Split_col = sorted(Split_col)
            prom=promedio(Split_col)
            if (len(prom)>0):
                Salary = Extract(lol, num_col_salary)
                error_min, error_max, valor = rss(prom, col, Salary)
                errores_min.append(error_min)
                errores_max.append(error_max)
                values_c.append(valor)
            
    # Minímo valor        
    min_error = min(errores_min)
    min_val_index = errores_min.index(min(errores_min))

    # Valor de corte del mínimo error
    corte_val = values_c[min_val_index]

    # Indice de la columna con el menor valor
    column_name = header[min_val_index]
    return min_error, min_val_index, corte_val

In [79]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [80]:
def build_tree_2(rows, poda):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    ###gain, question = find_best_split(rows)
 #   if(len(rows) > (poda-1)):
 #     min_error, num_col, c_value  = min_error, name_col, c_value = errores(rows, num_col_salary)
 #     gain = min_error
 #       question = Question(num_col, c_value)
 #   else:
 #       return Leaf(rows)
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 or len(rows) < poda:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    
    # Recursively build the true branch.
    true_branch = build_tree_2(true_rows, poda)

    # Recursively build the false branch.
    false_branch = build_tree_2(false_rows, poda)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [81]:
# Impresión del árbol
def print_tree2(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree2(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree2(node.false_branch, spacing + "  ")

In [82]:
# Clase hoja, guarda los valores de las etiquetas
# En el caso de regresión aquí van los promedios o determinar promedios
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [83]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

### Pruebas

In [84]:
num_col_salary = 16

In [85]:
#Con el campo años
Split_years = unique_vals(hitters_lol,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#Split_years
# Probando promedios entre valores
prom = promedio(Split_years)
prom
Salary = Extract(hitters_lol, num_col_salary)
#Salary
# El mejor dato de la división
error_min, error_max, val_corte = rss(prom, Extract(hitters_lol,6),Salary)
print("error min: " + str(error_min) + ", valor de corte: " + str(val_corte))

error min: 115.05847519916081, valor de corte: 4.5


In [86]:
#Con el campo hits
Split_hits = unique_vals(hitters_lol,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#Split_years
# Probando promedios entre valores
prom = promedio(Split_hits)
prom
Salary = Extract(hitters_lol, num_col_salary)
#Salary
# El mejor dato de la división
error_min, error_max, val_corte = rss(prom, Extract(hitters_lol,1),Salary)
print("error min: " + str(error_min) + ", valor de corte: " + str(val_corte))

error min: 160.9715298124983, valor de corte: 117.5


In [87]:
true_rows, false_rows = partition(hitters_lol, Question(6, 4.5))

In [88]:
#Con el campo hits para la rama derecha
Split_hits = unique_vals(true_rows,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#
Salary = Extract(true_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_hits)
#prom
rss(prom, Extract(true_rows,1),Salary)

(48.97678249450081, 72.6595406653801, 117.5)

In [89]:
#Con el campo years para la rama derecha
Split_years = unique_vals(true_rows,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#
Salary = Extract(true_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_years)
#prom
rss(prom, Extract(true_rows,6),Salary)

(69.87701889391077, 72.6800485480704, 6.5)

In [90]:
#Con el campo years para la rama izquierda
Split_years = unique_vals(false_rows,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#
Salary = Extract(false_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_years)
#prom
rss(prom, Extract(false_rows,6),Salary)

(33.14306582442718, 38.98184820582178, 3.5)

In [91]:
#Con el campo years para la rama izquierda
Split_hits = unique_vals(false_rows,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#
Salary = Extract(false_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_hits)
#prom
rss(prom, Extract(false_rows,1),Salary)

(33.0145870443409, 42.33821620845311, 15.5)

In [92]:
num_cols = len(hitters_lol[0]) 
num_cols

17

In [93]:
min_error, name_col, c_value = errores(hitters_lol, num_col_salary)
min_error

89.29612127664447

In [94]:
name_col

7

In [95]:
header[name_col]

'CAtBat'

In [96]:
c_value

1452.0

In [97]:
Question(7, c_value)

Is CAtBat >= 1452.0?

#### Probando para todo el conjunto con las variables Hits y Years

In [98]:
num_col_salary = 2
header = ["Hits","Years","Salary"]
poda = 50
my_tree = build_tree_2(hits_years_lol, poda)
print_tree2(my_tree)

Is Years >= 4.0?
--> True:
  Is Hits >= 93.0?
  --> True:
    Is Hits >= 137.0?
    --> True:
      Is Years >= 11.0?
      --> True:
        Predict {6.21460809842219: 2, 6.62007320653036: 2, 6.652863029353349: 1, 6.655011687076089: 1, 6.802394763324309: 1, 6.856461984594589: 1, 6.83876205035226: 1, 7.549609165154529: 1, 6.94857759350234: 1, 7.52911640503463: 1, 7.788418833213139: 1, 6.9077552789821395: 3, 7.49554194388426: 1, 6.263398262591621: 1, 6.551080335043401: 1, 7.279318835414621: 1}
      --> False:
        Predict {6.24831943200756: 1, 6.639875833826539: 1, 6.437751649736401: 2, 6.417548942418881: 1, 6.703188113240861: 1, 7.090076835776089: 1, 7.207859871432481: 1, 4.6539603501575195: 1, 7.588323677335219: 1, 7.807916628926411: 1, 7.069023426578259: 1, 6.802394763324309: 1, 5.99146454710798: 1, 6.496020557860381: 1, 6.856461984594589: 1, 7.120848494442849: 1, 5.52146091786225: 1, 6.652863029353349: 1, 5.625820933186491: 1, 6.48768401848461: 1, 6.461468176353719: 1, 6.4457198

#### Probando para variables continuas

In [39]:
num_col_salary = 16
# Column labels.
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors","Salary"]

In [40]:
poda = 80
my_tree = build_tree_2(hitters_lol, poda)
print_tree2(my_tree)

Is CAtBat >= 1452.0?
--> True:
  Is Hits >= 117.5?
  --> True:
    Is CRBI >= 273.0?
    --> True:
    --> False:
  --> False:
--> False:
  Is CHits >= 182.0?
  --> True:
  --> False:
